In [ ]:
import os
import pandas as pd
from docx import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 文件夹路径
folder_path = ""
output_folder_path = folder_path  # 输出CSV文件夹路径（保持不变）

# 确保输出文件夹存在
os.makedirs(output_folder_path, exist_ok=True)

# 自定义分割器：优先按标点符号分割
splitter = RecursiveCharacterTextSplitter(
    separators=["。", "！", "？", "\n\n", "\n", " ", ""],  # 优先按标点符号分割
    chunk_size=1000,
    chunk_overlap=0  # 无重叠
)

# 定义函数：清理段落开头的标签符号
def clean_paragraphs(chunks):
    cleaned_chunks = []
    for chunk in chunks:
        # 去除段落开头的空格、换行符以及多余标点符号
        cleaned_chunk = chunk.lstrip(" \n。！？；，")
        cleaned_chunks.append(cleaned_chunk)
    return cleaned_chunks

# 获取文件夹中的所有docx文件
docx_files = [f for f in os.listdir(folder_path) if f.endswith('.docx')]

# 逐个处理文件
for docx_file in docx_files:
    # 构造文件路径
    docx_file_path = os.path.join(folder_path, docx_file)

    # 读取.docx文件内容
    document = Document(docx_file_path)
    text = "\n".join([para.text for para in document.paragraphs if para.text.strip()])  # 合并段落并去除空白段落

    # 分割文本
    chunks = splitter.split_text(text)

    # 清理段落开头的符号或空格
    chunks = clean_paragraphs(chunks)

    # 构造输出CSV文件路径，保持文件名不变
    csv_file_name = f"{os.path.splitext(docx_file)[0]}.csv"
    csv_file_path = os.path.join(output_folder_path, csv_file_name)

    # 将分割后的文本保存到CSV文件中
    df = pd.DataFrame({'文本段': chunks})
    df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

    print(f"文件 {docx_file} 的分割结果已保存到 {csv_file_path}")
